In [14]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
    
root_dir='/gpfs/fs1/work/hongli/sharp/basins/tayprk_huc12/scripts_hongli/'
os.chdir(root_dir)
output_folder='step6_runoff_ratio'
text_outputfile = 'runoff_ratio.txt'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

summa_file = '/gpfs/fs1/work/hongli/sharp/basins/tayprk_huc12/output/gmet_3hr/output_output_day.nc'
route_file = '/gpfs/fs1/work/hongli/sharp/basins/tayprk_huc12/route/output/all.nc'
obs_file ='Taylor_park_09107000.txt'

p_vname = 'pptrate_mean' #'kg m-2 s-1'
t_vname = 'time'
qirf_vname = 'IRFroutedRunoff'
time_format='%Y-%m-%d'
huc_area = np.asarray([74651366.29, 96123140.3, 161342963.5]) #m^2

# read precip.(mm/d)
print('read')
f = xr.open_dataset(summa_file)
var_list = f.variables
sim_time_str = f[t_vname].dt.strftime(time_format).values[:]
sim_time = np.asarray([datetime.datetime.strptime(t, time_format) for t in sim_time_str])
wateryear_list = np.unique([t.year for t in sim_time])[1:]
p_value = f[p_vname].values[:]
f.close()

# read simulated flow (m^3/s)
f = xr.open_dataset(route_file)
sim_irf = f[qirf_vname].values[:]
f.close()

# read observed flow (f^3/s)
obs_read = []
obs_time_str = []
with open(obs_file) as f:
    for line in f:
        line = line.strip()
        if line and line.startswith('USGS'):
            splits = line.split()
            obs_time_str.append(splits[2])
            if len(splits)>3:#flow data exists
                obs_read.append(float(splits[3]))
            else:
                obs_read.append(np.nan)
obs_read = np.asarray(obs_read)
obs_time = np.asarray([datetime.datetime.strptime(t, time_format) for t in obs_time_str])
                
# calculate
print('calculate')

details = []
for wateryear in wateryear_list:

    # identify wateryear corresponding index
    wateryear_ini = datetime.datetime.strptime(str(wateryear-1)+'-10-01', time_format)
    wateryear_end = datetime.datetime.strptime(str(wateryear)+'-09-30', time_format)        
    sim_wateryear_index=[loc for loc in range(len(sim_time)) if (sim_time[loc]>= wateryear_ini and sim_time[loc]<=wateryear_end)]
    obs_wateryear_index=[loc for loc in range(len(obs_time)) if (obs_time[loc]>= wateryear_ini and obs_time[loc]<=wateryear_end)]

    # calcualte annual cumulative flux
    p = np.sum(np.nansum(p_value[sim_wateryear_index,:], axis=0)*86400*0.001*huc_area) # mm/s to mm/day to m^3/day
    sim_runoff = np.nansum(sim_irf[sim_wateryear_index])*86400 # m^3/s to m^3/day
    obs_runoff = np.nansum(obs_read[obs_wateryear_index])*86400/35.315 # ft^3/s to m^3/day

    # runoff ratio
    sim_ratio = sim_runoff/p *100 #percentage
    obs_ratio = obs_runoff/p *100
    
    # save annual data
    hru_year_data = [wateryear, p, sim_runoff, obs_runoff, sim_ratio, obs_ratio]
    details.append(hru_year_data)

# output
print('save')
header_txt='Water year, P(m^3), Sim_Runoff(m^3), Obs_Runoff(m^3), Sim_ratio(%), Obs_ratio(%)' 
np.savetxt(os.path.join(root_dir, output_folder, text_outputfile), details, header=header_txt, fmt='%d, %.4f, %.4f, %.4f, %.2f, %.2f')


read
calculate
save


In [15]:
np.savetxt('obs_flow.txt', obs_read/35.315, delimiter=',', fmt='%.2f')